#Text-to-scene Generation Testing playground.
###The notebook is in debugging mode and may, occasionally, present errors. 
###currently under research at Indian Institute of Technology, Hyderabad.

In [ ]:
import subprocess, os, sys
sub_p_res = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total,memory.free', '--format=csv,noheader'], stdout=subprocess.PIPE).stdout.decode('utf-8')
print(f"{sub_p_res[:-1]}")

# Setup

In [ ]:

import subprocess, time, gc, os, sys

def setup_environment():
    print_subprocess = False
    use_xformers_for_colab = True
    try:
        ipy = get_ipython()
    except:
        ipy = 'could not get_ipython'
    if 'google.colab' in str(ipy):
        print("..setting up environment")
        start_time = time.time()
        all_process = [
            ['pip', 'install', 'torch==1.12.1+cu113', 'torchvision==0.13.1+cu113', '--extra-index-url', 'https://download.pytorch.org/whl/cu113'],
            ['pip', 'install', 'omegaconf==2.2.3', 'einops==0.4.1', 'pytorch-lightning==1.7.4', 'torchmetrics==0.9.3', 'torchtext==0.13.1', 'transformers==4.21.2', 'kornia==0.6.7'],
            ['git', 'clone', 'https://github.com/deforum-art/deforum-stable-diffusion'],
            ['pip', 'install', 'accelerate', 'ftfy', 'jsonmerge', 'matplotlib', 'resize-right', 'timm', 'torchdiffeq','scikit-learn'],
        ]
        for process in all_process:
            running = subprocess.run(process,stdout=subprocess.PIPE).stdout.decode('utf-8')
            if print_subprocess:
                print(running)
        with open('deforum-stable-diffusion/src/k_diffusion/__init__.py', 'w') as f:
            f.write('')
        sys.path.extend([
            'deforum-stable-diffusion/',
            'deforum-stable-diffusion/src',
        ])
        end_time = time.time()

        if use_xformers_for_colab:

            print("..installing xformers")

            all_process = [['pip', 'install', 'triton==2.0.0.dev20220701']]
            for process in all_process:
                running = subprocess.run(process,stdout=subprocess.PIPE).stdout.decode('utf-8')
                if print_subprocess:
                    print(running)
                    
            v_card_name = subprocess.run(['nvidia-smi', '--query-gpu=name', '--format=csv,noheader'], stdout=subprocess.PIPE).stdout.decode('utf-8')
            if 't4' in v_card_name.lower():
                name_to_download = 'T4'
            elif 'v100' in v_card_name.lower():
                name_to_download = 'V100'
            elif 'a100' in v_card_name.lower():
                name_to_download = 'A100'
            elif 'p100' in v_card_name.lower():
                name_to_download = 'P100'
            else:
                print(v_card_name + ' is currently not supported with xformers flash attention in deforum!')

            x_ver = 'xformers-0.0.13.dev0-py3-none-any.whl'
            x_link = 'https://github.com/TheLastBen/fast-stable-diffusion/raw/main/precompiled/' + name_to_download + '/' + x_ver
        
            all_process = [
                ['wget', x_link],
                ['pip', 'install', x_ver],
                ['mv', 'deforum-stable-diffusion/src/ldm/modules/attention.py', 'deforum-stable-diffusion/src/ldm/modules/attention_backup.py'],
                ['mv', 'deforum-stable-diffusion/src/ldm/modules/attention_xformers.py', 'deforum-stable-diffusion/src/ldm/modules/attention.py']
            ]

            for process in all_process:
                running = subprocess.run(process,stdout=subprocess.PIPE).stdout.decode('utf-8')
                if print_subprocess:
                    print(running)

            print(f"Environment set up in {end_time-start_time:.0f} seconds")
    else:
        sys.path.extend([
            'src'
        ])
    return

setup_environment()

import torch
import random
import clip
from IPython import display
from types import SimpleNamespace
from helpers.save_images import get_output_folder
from helpers.settings import load_args
from helpers.render import render_animation, render_input_video, render_image_batch, render_interpolation
from helpers.model_load import make_linear_decode, load_model, get_model_output_paths
from helpers.aesthetics import load_aesthetics_model


def Root():
    models_path = "models" 
    configs_path = "configs" 
    output_path = "output" 
    mount_google_drive = True #
    models_path_gdrive = "/content/drive/MyDrive/AI/models" 
    output_path_gdrive = "/content/drive/MyDrive/AI/StableDiffusion" 

    model_config = "v1-inference.yaml" 
    model_checkpoint =  "v1-5-pruned-emaonly.ckpt" 
    custom_config_path = "" 
    custom_checkpoint_path = "" 
    half_precision = True
    return locals()

root = Root()
root = SimpleNamespace(**root)

root.models_path, root.output_path = get_model_output_paths(root)
root.model, root.device = load_model(root, 
                                    load_on_run_all=True
                                    , 
                                    check_sha256=True
                                    )

# Settings

In [ ]:
def DeforumAnimArgs():

   
    animation_mode = 'None' 
    max_frames = 1000 
    border = 'replicate' 

    angle = "0:(0)"
    zoom = "0:(1.04)"
    translation_x = "0:(10*sin(2*3.14*t/10))"
    translation_y = "0:(0)"
    translation_z = "0:(10)"
    rotation_3d_x = "0:(0)"
    rotation_3d_y = "0:(0)"
    rotation_3d_z = "0:(0)"
    flip_2d_perspective = False 
    perspective_flip_theta = "0:(0)"
    perspective_flip_phi = "0:(t%15)"
    perspective_flip_gamma = "0:(0)"
    perspective_flip_fv = "0:(53)"
    noise_schedule = "0: (0.02)"
    strength_schedule = "0: (0.65)"
    contrast_schedule = "0: (1.0)"

    color_coherence = 'Match Frame 0 LAB' 
    diffusion_cadence = '1' 

    use_depth_warping = True
    midas_weight = 0.3
    near_plane = 200
    far_plane = 10000
    fov = 40
    padding_mode = 'border'
    sampling_mode = 'bicubic'
    save_depth_maps = False 

    video_init_path ='/content/video_in.mp4'
    extract_nth_frame = 1
    overwrite_extracted_frames = True 
    use_mask_video = False 
    video_mask_path ='/content/video_in.mp4'

    interpolate_key_frames = False 
    interpolate_x_frames = 4 
    
    resume_from_timestring = False 
    resume_timestring = "20220829210106" 

    return locals()

In [ ]:
prompts = [
    "A marvellours painting of a woman by Da Vinci, sitting on the patio with the sunset in the background", 
    "An wonderful painting of a tall woman by Da Vinci, sitting on the sofa with the sunset in the background", 
    
]

animation_prompts = {
    0: "a beautiful woman, sitting on the patio,
    20: "a beautiful woman, standing on the patio",
    30: "a beautiful woman, stretching on the patio",
    40: "a beautiful woman, sleeping on the patio",
}

In [ ]:
override_settings_with_file = False 
settings_file = "custom"
custom_settings_file = "/content/drive/MyDrive/Settings.txt"

def DeforumArgs():
    
    W = 512 
    H = 512
    W, H = map(lambda x: x - x % 64, (W, H))  

    seed = -1
    sampler = 'dpmpp_2s_a' 
    steps = 80 
    scale = 7 
    ddim_eta = 0.0
    dynamic_threshold = None
    static_threshold = None   

    save_samples = True 
    save_settings = True 
    display_samples = True 
    save_sample_per_step = False 
    show_sample_per_step = False 

    prompt_weighting = True 
    normalize_prompt_weights = True 
    log_weighted_subprompts = False 

    n_batch = 1 
    batch_name = "StableFun" 
    filename_format = "{timestring}_{index}_{prompt}.png" 
    seed_behavior = "iter"
    make_grid = False
    grid_rows = 2 
    outdir = get_output_folder(root.output_path, batch_name)

    use_init = False 
    strength = 0.0 
    strength_0_no_init = True 
    init_image = "https://cdn.pixabay.com/photo/2022/07/30/13/10/green-longhorn-beetle-7353749_1280.jpg" 
   
    use_mask = False 
    use_alpha_as_mask = False 
    mask_file = "https://www.filterforge.com/wiki/images/archive/b/b7/20080927223728%21Polygonal_gradient_thumb.jpg" 
    invert_mask = False 
    mask_brightness_adjust = 1.0  
    mask_contrast_adjust = 1.0  
    overlay_mask = True 
    mask_overlay_blur = 5 
    mean_scale = 0 
    var_scale = 0 
    exposure_scale = 0 
    exposure_target = 0.5 

    colormatch_scale = 0 
    colormatch_image = "https://www.saasdesign.io/wp-content/uploads/2021/02/palette-3-min-980x588.png" 
    colormatch_n_colors = 4 
    ignore_sat_weight = 0 
    clip_name = 'ViT-L/14' 
    clip_scale = 0 
    aesthetics_scale = 0 
    cutn = 1 
    cut_pow = 0.0001 

    init_mse_scale = 0 
    init_mse_image = "https://cdn.pixabay.com/photo/2022/07/30/13/10/green-longhorn-beetle-7353749_1280.jpg" 
    
    gradient_wrt = 'x0_pred' 
    gradient_add_to = 'both' 
    decode_method = 'linear' 
    grad_threshold_type = 'dynamic' 
    clamp_grad_threshold = 0.2 
    clamp_start = 0.2 
    clamp_stop = 0.01 
    grad_inject_timing = list(range(1,10)) 

    cond_uncond_sync = True 

    n_samples = 1 
    precision = 'autocast' 
    C = 4
    f = 8

    prompt = ""
    timestring = ""
    init_latent = None
    init_sample = None
    init_sample_raw = None
    mask_sample = None
    init_c = None

    return locals()

args_dict = DeforumArgs()
anim_args_dict = DeforumAnimArgs()

if override_settings_with_file:
    load_args(args_dict, anim_args_dict, settings_file, custom_settings_file, verbose=False)

args = SimpleNamespace(**args_dict)
anim_args = SimpleNamespace(**anim_args_dict)

args.timestring = time.strftime('%Y%m%d%H%M%S')
args.strength = max(0.0, min(1.0, args.strength))

if (args.clip_scale > 0) or (args.aesthetics_scale > 0):
    root.clip_model = clip.load(args.clip_name, jit=False)[0].eval().requires_grad_(False).to(root.device)
    if (args.aesthetics_scale > 0):
        root.aesthetics_model = load_aesthetics_model(args, root)

if args.seed == -1:
    args.seed = random.randint(0, 2**32 - 1)
if not args.use_init:
    args.init_image = None
if args.sampler == 'plms' and (args.use_init or anim_args.animation_mode != 'None'):
    print(f"Init images aren't supported with PLMS yet, switching to KLMS")
    args.sampler = 'klms'
if args.sampler != 'ddim':
    args.ddim_eta = 0

if anim_args.animation_mode == 'None':
    anim_args.max_frames = 1
elif anim_args.animation_mode == 'Video Input':
    args.use_init = True

gc.collect()
torch.cuda.empty_cache()

if anim_args.animation_mode == '2D' or anim_args.animation_mode == '3D':
    render_animation(args, anim_args, animation_prompts, root)
elif anim_args.animation_mode == 'Video Input':
    render_input_video(args, anim_args, animation_prompts, root)
elif anim_args.animation_mode == 'Interpolation':
    render_interpolation(args, anim_args, animation_prompts, root)
else:
    render_image_batch(args, prompts, root)

# Create Video From Frames

In [ ]:
skip_video_for_run_all = True 
fps = 12 
use_manual_settings = False
image_path = "/content/drive/MyDrive/AI/StableDiffusion/2022-09/20220903000939_%05d.png" #@param {type:"string"}
mp4_path = "/content/drive/MyDrive/AI/StableDiffusion/2022-09/20220903000939.mp4" #@param {type:"string"}
render_steps = False 
path_name_modifier = "x0_pred" 
make_gif = False

if skip_video_for_run_all == True:
    print('Skipping video creation, uncheck skip_video_for_run_all if you want to run it')
else:
    import os
    import subprocess
    from base64 import b64encode

    print(f"{image_path} -> {mp4_path}")

    if use_manual_settings:
        max_frames = "200" 
    else:
        if render_steps: 
            fname = f"{path_name_modifier}_%05d.png"
            all_step_dirs = [os.path.join(args.outdir, d) for d in os.listdir(args.outdir) if os.path.isdir(os.path.join(args.outdir,d))]
            newest_dir = max(all_step_dirs, key=os.path.getmtime)
            image_path = os.path.join(newest_dir, fname)
            print(f"Reading images from {image_path}")
            mp4_path = os.path.join(newest_dir, f"{args.timestring}_{path_name_modifier}.mp4")
            max_frames = str(args.steps)
        else:
            image_path = os.path.join(args.outdir, f"{args.timestring}_%05d.png")
            mp4_path = os.path.join(args.outdir, f"{args.timestring}.mp4")
            max_frames = str(anim_args.max_frames)

    cmd = [
        'ffmpeg',
        '-y',
        '-vcodec', 'png',
        '-r', str(fps),
        '-start_number', str(0),
        '-i', image_path,
        '-frames:v', max_frames,
        '-c:v', 'libx264',
        '-vf',
        f'fps={fps}',
        '-pix_fmt', 'yuv420p',
        '-crf', '17',
        '-preset', 'veryfast',
        '-pattern_type', 'sequence',
        mp4_path
    ]
    process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    if process.returncode != 0:
        print(stderr)
        raise RuntimeError(stderr)

    mp4 = open(mp4_path,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    display.display(display.HTML(f'<video controls loop><source src="{data_url}" type="video/mp4"></video>') )
    
    if make_gif:
         gif_path = os.path.splitext(mp4_path)[0]+'.gif'
         cmd_gif = [
             'ffmpeg',
             '-y',
             '-i', mp4_path,
             '-r', str(fps),
             gif_path
         ]
         process_gif = subprocess.Popen(cmd_gif, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

In [ ]:
skip_disconnect_for_run_all = True 

if skip_disconnect_for_run_all == True:
    print('Skipping disconnect, uncheck skip_disconnect_for_run_all if you want to run it')
else:
    from google.colab import runtime
    runtime.unassign()